<a href="https://colab.research.google.com/github/vnavya2004/BTP/blob/main/Paper(1_s2_0_S0957417422020255_main)_Arabic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Step 1: Install Required Libraries
!pip install numpy pandas tensorflow keras gensim nltk scikit-learn openpyxl


In [2]:

# Step 2: Import Libraries
import numpy as np
import pandas as pd
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Conv1D, MaxPooling1D, Dense, Dropout, Input, Bidirectional, Attention, GlobalAveragePooling1D, Concatenate
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [114]:


# Step 3: Load and Preprocess Data
# Load the Excel file
from google.colab import files
uploaded = files.upload()  # Manually upload your .xlsx file

# Change the file name as per the uploaded file


Saving Arabic_Depression_10.000_Tweets_translated_bangla.xlsx to Arabic_Depression_10.000_Tweets_translated_bangla (1).xlsx


In [115]:
df = pd.read_excel(next(iter(uploaded.keys())))  # Replace with 'filename.xlsx' if needed

In [116]:
# df = df.sample(frac=0.4, random_state=42)
print(df)

      Unnamed: 0                                              tweet  label  \
0              0  والله شايفيتلك الناس كلها صاير عندها اكتئاب لا...      1   
1              1  اكيد اكتئاب بعد العصر ما في زيه بس ان شاء الله...      1   
2              2  والله ما أنا عارف هو اكتئاب ولا انطفاء ولا نضج...      1   
3              3                     اصبروا داخله في حاله اكتئاب 💔😭      1   
4              4                                     هذا اكتئاب ولا      1   
...          ...                                                ...    ...   
9995        5034                             من الناس الحكيمة جدا.       0   
9996        5035  ساكون افضل مما اكون وادع الناس يقولون ما يقولو...      0   
9997        5036  يا سلام علي لحظه الاستجمام و الاستماع لام كلثوم.       0   
9998        5037                                  دى اسرع ريتويت .       0   
9999        5038                  وانتى وكل اللى فى المنشن طيبين .       0   

                                           tweet_bangla  
0    

In [117]:

# Define the text and label columns
text_column_name = 'tweet_bangla'  # replace with actual column name for text
label_column_name = 'label'  # replace with actual column name for labels


In [118]:
df[text_column_name]

,tweet_bangla
0,"খোদার শপথ করে বলছি, এই সব লোকের মন খারাপ হয়ে ..."
1,"অবশ্য বিকেলের পর বিষণ্নতা বলে কিছু নেই, তবে ইন..."
2,"ঈশ্বরের কসম, আমি জানি না এটি হতাশা, বিলুপ্তি, ..."
3,তিনি বিষণ্ণ অবস্থায় থাকা অবস্থায় ধৈর্য ধরুন 💔😭
4,এটা বিষণ্নতা নয়
...,...
9995,খুব জ্ঞানী মানুষ।
9996,আমি আমার চেয়ে ভাল হব এবং লোকেরা যা বলে তা বলত...
9997,"ওহ, উম্মে কুলথুমের কথা শুনে আরাম করার কী মুহূর্ত।"
9998,এটি দ্রুততম রিটুইট।


In [119]:

# Preprocess text data
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

def preprocess_text(text):
    # Tokenization
    # print(text)
    tokens = word_tokenize(str(text).lower())
    # print("Tokens:", tokens)  # Debugging line

    # Filter out non-alphabetic tokens
    # words = [word for word in tokens if word.isalpha()]
    # print("After non-alphabetic filtering:", words)  # Debugging line

    # Remove stopwords
    words = [word for word in tokens]  # Keep all tokens for now
    # print("After initial filtering:", words)  # Debugging line

    words = [word for word in words if word not in stopwords.words('bengali')]
    # print("After stopword removal:", words)  # Debugging line

    # Join back into a single string
    return " ".join(words)


df[text_column_name] = df[text_column_name].apply(preprocess_text)

# Step 4: Split Data into Train and Test Sets
X_train, X_test, y_train, y_test = train_test_split(df[text_column_name], df[label_column_name], test_size=0.2, random_state=42)


In [111]:
print(X_train.head)

<bound method NDFrame.head of 637     প্রতীক একদিন দুর্বল ছিলাম , চিহ্ন সর্বদা সাথে ...
478     বিষণ্নতা মরার ইচ্ছা , জানি বছর পড়াশোনা শক্তি ...
256                                         কোথাও ফিট নই।
1486    শান্ত , স্ক্র্যাপারের র‍্যাপ বর্ণবাদীর রেস , শীতল
440              নিখুঁত বোধ পরাজিত ... হঠাৎ একটা ঘটে করুণ
                              ...                        
1638    আজকের রাতটি সাহসী , দীর্ঘ স্ত্রী তিনটি বাচ্চা ...
1095    সম্ভবত আসবে , কুকুর , উইনি , আবারও পালিয়ে , 5...
1130    ফিটনেস ক্লাসে পিপি সমস্যা বাচ্চাদের ভাষা ফিল্ট...
1294    হাই স্কুল প্রত্যেকের , জানি সম্ভবত শুনেছেন , আ...
860                                                 হতাশা
Name: Tweets_bangla, Length: 1748, dtype: object>


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [72]:

fasttext_model = gensim.models.KeyedVectors.load('/content/drive/MyDrive/embeddings/fasttext_bn_model.kv')


In [22]:
from gensim.models import KeyedVectors
from google.colab import drive
from gensim.scripts.glove2word2vec import glove2word2vec

# # Load the GloVe embeddings
# glove_model_path = '/content/drive/MyDrive/bn_glove.39M.300d.txt'
# word2vec_output_file = '/content/GloVe-Bengali/word2vec_bengali.txt'

# # Convert GloVe format to Word2Vec format
# glove2word2vec(glove_model_path, word2vec_output_file)



In [20]:
# glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)


In [21]:

# # Save the model to Google Drive in binary format
# glove_model_save_path = '/content/drive/MyDrive/embeddings/glove_bn_model.kv'
# os.makedirs(os.path.dirname(glove_model_save_path), exist_ok=True)

# glove_model.save(glove_model_save_path)

In [22]:
glove_model_path = '/content/drive/MyDrive/embeddings/glove_bn_model.kv'
glove_model = KeyedVectors.load(glove_model_path)

In [25]:
# !git clone https://github.com/sagorbrur/bnlp.git
# #

Cloning into 'bnlp'...
remote: Enumerating objects: 2130, done.
remote: Counting objects: 100% (653/653), done.
remote: Compressing objects: 100% (322/322), done.
remote: Total 2130 (delta 387), reused 535 (delta 326), pack-reused 1477 (from 1)
Receiving objects: 100% (2130/2130), 22.76 MiB | 26.10 MiB/s, done.
Resolving deltas: 100% (1265/1265), done.


In [27]:
!pip install bnlp_toolkit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 43.3 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171033 sha256=abf60055a10b156168e1b94df5d796a5b1a810fc05b73854a4f69a697374be02
  Stored in directory: /root/.cache/pip/wheels/31/8a/8c/315c9e5d7773f74b33d5ed33f075b49c6eaeb7cedbb86e2cf8
Successfully built emoji
  Attempting uninstall: tqdm
    Found exist

In [120]:
from bnlp import BengaliWord2Vec
from gensim.models import KeyedVectors
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, Input, Concatenate
from tensorflow.keras.models import Model

# Initialize BengaliWord2Vec with pretrained model
bwv = BengaliWord2Vec()

# Define function to retrieve word vector embeddings for BengaliWord2Vec
def get_bwv_embedding_matrix(word_index):
    """
    Creates an embedding matrix for the BengaliWord2Vec model.

    Args:
        word_index (dict): A dictionary mapping words to their indices.

    Returns:
        np.ndarray: The embedding matrix.
    """
    embedding_dim = 100  # Assuming a dimension of 300 for BengaliWord2Vec
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        try:
            vector = bwv.get_word_vector(word)
            if vector is not None and len(vector) > 0:
                embedding_matrix[i] = vector
        except KeyError:
            print(f"Warning: Word '{word}' not found in the BengaliWord2Vec vocabulary. Skipping.")
    return embedding_matrix

# Define function to retrieve word vector embeddings for other models
def get_embedding_matrix(word_index, embedding_model, embedding_dim=300):
    """
    Creates an embedding matrix for a given embedding model.

    Args:
        word_index (dict): A dictionary mapping words to their indices.
        embedding_model (object): The word embedding model.
        embedding_dim (int): The desired embedding dimension.

    Returns:
        np.ndarray: The embedding matrix.
    """
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in embedding_model:
            embedding_matrix[i] = embedding_model[word]
    return embedding_matrix

# Tokenize the input data
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(X_train)

train_sequences = tokenizer.texts_to_sequences(X_train)
test_sequences = tokenizer.texts_to_sequences(X_test)

padded_train = pad_sequences(train_sequences, maxlen=100)
padded_test = pad_sequences(test_sequences, maxlen=100)
word_index = tokenizer.word_index

# Step 4: Create embedding matrices for each model
embedding_matrix_bwv = get_bwv_embedding_matrix(word_index)
embedding_matrix_ft = get_embedding_matrix(word_index, fasttext_model, embedding_dim=300)
embedding_matrix_glove = get_embedding_matrix(word_index, glove_model, embedding_dim=300)

# Step 5: Build the Hybrid Model
# Input layer
input_layer = Input(shape=(100,))

# Embedding layers for each embedding type
embedding_layer_bwv = Embedding(len(word_index) + 1, 100, weights=[embedding_matrix_bwv], trainable=False)(input_layer)
embedding_layer_ft = Embedding(len(word_index) + 1, 300, weights=[embedding_matrix_ft], trainable=False)(input_layer)
embedding_layer_glove = Embedding(len(word_index) + 1, 300, weights=[embedding_matrix_glove], trainable=False)(input_layer)

# Concatenate embeddings
concatenated_embeddings = Concatenate()([embedding_layer_bwv, embedding_layer_ft, embedding_layer_glove])


# Continue building the model as needed...


In [82]:
print(concatenated_embeddings)

<KerasTensor shape=(None, 100, 700), dtype=float32, sparse=False, name=keras_tensor_48>


In [70]:
# Check dimension for each model to ensure compatibility
fasttext_dim = len(fasttext_model[next(iter(fasttext_model.key_to_index))])  # For gensim FastText model
glove_dim = len(glove_model[next(iter(glove_model.key_to_index))])          # For gensim GloVe model

# Verify dimensions
print(f"FastText embedding dimension: {fasttext_dim}")
print(f"GloVe embedding dimension: {glove_dim}")

FastText embedding dimension: 300
GloVe embedding dimension: 300


In [121]:
from keras.models import Model
from keras.layers import Input, Bidirectional, LSTM, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Attention
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

# Assuming `input_layer` and `concatenated_embeddings` are already defined
# Layer definitions
bilstm_out = Bidirectional(LSTM(100, return_sequences=True))(concatenated_embeddings)
lstm_out = LSTM(100, return_sequences=True)(bilstm_out)
cnn_out = Conv1D(filters=50, kernel_size=3, activation='relu')(lstm_out)

attention_out = Attention()([cnn_out, cnn_out])  # Adjusted for attention layer

# Global max pooling and dense layers
max_pool_out = GlobalMaxPooling1D()(attention_out)  # Applying GlobalMaxPooling1D after attention
dropout_layer_1 = Dropout(0.5)(max_pool_out)
dense_layer_1 = Dense(250, activation='relu')(dropout_layer_1)
dropout_layer_2 = Dropout(0.5)(dense_layer_1)
output_layer = Dense(1, activation='sigmoid')(dropout_layer_2)  # Use sigmoid for binary classification

# Define and compile model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])  # Use binary crossentropy for binary classification
model.summary()

# Step 8: Train the Model
model.fit(padded_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Step 9: Evaluate the Model
predictions = model.predict(padded_test)
predicted_classes = (predictions > 0.5).astype("int32")  # Get the predicted class indices based on a threshold
print("Accuracy:", accuracy_score(y_test, predicted_classes))
print("AUC:", roc_auc_score(y_test, predictions))  # For binary classification, use the predictions directly
print("Classification Report:\n", classification_report(y_test, predicted_classes))


Model: "functional_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_8             │ (None, 100)            │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_17 (Embedding)  │ (None, 100, 100)       │        964,100 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_18 (Embedding)  │ (None, 100, 300)       │      2,892,300 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ embedding_19 (Embedding)  │ (None, 100, 300)       │      2,892,300 │ input_layer_8[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_5             │ (None, 100, 700)       │              0 │ embedding_17[0][0],    │
│ (Concatenate)             │                        │                │ embedding_18[0][0],    │
│                           │                        │                │ embedding_19[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ bidirectional_5           │ (None, 100, 200)       │        640,800 │ concatenate_5[0][0]    │
│ (Bidirectional)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_11 (LSTM)            │ (None, 100, 100)       │        120,400 │ bidirectional_5[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1d_5 (Conv1D)         │ (None, 98, 50)         │         15,050 │ lstm_11[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attention_5 (Attention)   │ (None, 98, 50)         │              0 │ conv1d_5[0][0],        │
│                           │                        │                │ conv1d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ global_max_pooling1d_5    │ (None, 50)             │              0 │ attention_5[0][0]      │
│ (GlobalMaxPooling1D)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_10 (Dropout)      │ (None, 50)             │              0 │ global_max_pooling1d_… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_10 (Dense)          │ (None, 250)            │         12,750 │ dropout_10[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout_11 (Dropout)      │ (None, 250)            │              0 │ dense_10[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_11 (Dense)          │ (None, 1)              │            251 │ dropout_11[0][0]       │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 7,537,951 (28.76 MB)

 Trainable params: 789,251 (3.01 MB)

 Non-trainable params: 6,748,700 (25.74 MB)

Epoch 1/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 7s 21ms/step - accuracy: 0.7685 - loss: 0.4661 - val_accuracy: 0.9100 - val_loss: 0.2001
Epoch 2/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9358 - loss: 0.1873 - val_accuracy: 0.9438 - val_loss: 0.1404
Epoch 3/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9582 - loss: 0.1208 - val_accuracy: 0.9425 - val_loss: 0.1671
Epoch 4/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9687 - loss: 0.0917 - val_accuracy: 0.9413 - val_loss: 0.1492
Epoch 5/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.9815 - loss: 0.0609 - val_accuracy: 0.9362 - val_loss: 0.2326
Epoch 6/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 6s 23ms/step - accuracy: 0.9785 - loss: 0.0555 - val_accuracy: 0.9488 - val_loss: 0.2237
Epoch 7/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 19ms/step - accuracy: 0.9903 - loss: 0.0300 - val_accuracy: 0.9475 - val_loss: 0.2187
Epoch 8/10
225/225 ━━━━━━━━━━━━━━━━━━━━ 4s 20ms/step - accuracy: 0.9879 - loss: 0.0336 - val_accu